# Third Assignment

##MSc course in Artificial Intelligence

insert your "Krunal Rathod"

In [ ]:
# !rm -r AI_USI_MA/
!git clone https://github.com/UmbertoJr/AI_USI_MA.git

In [ ]:
# Imports
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from AI_USI_MA.IO_manager.io_tsp import TSP_Instance_Creator
from AI_USI_MA.solvers.two_opt_with_candidate import twoOpt_with_cl
# if you are running from your local remove the prefix AI2020 (comment the previous line and uncomment the following line)
# from IO_manager.io_tsp import TSP_Instance_Creator

ic = TSP_Instance_Creator("standard", 'eil76.tsp')
ic.print_info()
ic.plot_data()

In [ ]:
ic = TSP_Instance_Creator("standard", 'ch130.tsp')
ic.print_info()
ic.plot_data()

In [ ]:
ic = TSP_Instance_Creator("standard", 'd198.tsp')
ic.print_info()
ic.plot_data()

In [ ]:
ic = TSP_Instance_Creator("standard", 'myTSP_dim10.tsp')
ic.print_info()
ic.plot_data()

In [ ]:
from AI_USI_MA.solvers.local_search import twoOpt
from AI_USI_MA.solvers.constructive_algorithms import nn

# nn takes as input the distance matrix and returns
# the tour and the length constructed with nearest neighbor, i.e.   tour, len_t = nn(dist_mat)

# twoOpt takes as input the solution, the actual_len and the distance matrix
# and returns the tour and the length created with 2-opt, i.e.     tour, lent_t = twoOpt(solution, actual_len, dist_mat)

class ACS:
  m = 10
  beta = 2
  alpha = rho = 0.1
  cl = 20 # Or 15

  @staticmethod
  def take_candidates(j, dist_mat, n):
    return list(np.argsort(dist_mat[j])[1:n+1])

  def __init__(self, instance):
    self.n = instance.nPoints
    self.dist_mat = instance.dist_matrix
    _, self.L_nn = nn(instance.dist_matrix, starting_node=np.random.choice(self.n))
    self.tau0 = 1./(float(self.n) * self.L_nn)
    self.position = {i: None for i in range(ACS.m)}  # position collector for the Ants, TO BE UPDATED during the steps
    self.tour = {i: [] for i in range(ACS.m)}  # tour collector for the Ants
    self.pheromone = {r: [self.tau0]*(self.n-1) for r in range(self.n)}
    self.candidate_list = {r: ACS.take_candidates(r, instance.dist_matrix, self.n) for r in range(self.n)}
    self.eta = {r: [1/self.dist_mat[r, s] for s in ACS.take_candidates(r, instance.dist_matrix, self.n)] for r in range(self.n)}

  def local_updating_rule(self, r, s):
    return (1 - self.rho) * self.pheromone[r][s] + self.rho * self.tau0

  def global_updating_rule(self, r, s, l_gb, tour_l_gb):
    to_index = self.candidate_list[r].index(s)
    from_index = self.candidate_list[s].index(r)

    i = tour_l_gb.index(r)
    j = tour_l_gb.index(s)
    if i + 1 == j or i - 1 == j:
      delta = 1.0/l_gb
    else:
      delta = 0

    self.pheromone[r][to_index] = (1 - self.alpha) * self.pheromone[r][to_index] + self.alpha * delta
    self.pheromone[s][from_index] = (1 - self.alpha) * self.pheromone[s][from_index] + self.alpha * delta

  def state_transition_rule(self, ant, r, q0):
    # Create the list of city already visited
    city_to_visit = []
    for c in self.candidate_list[r]:
      if not c in self.tour[ant]:
        city_to_visit.append(c)
      if len(city_to_visit) >= self.cl:
        break

    # Compute the exploitation array
    t = []
    for city in city_to_visit:
      i = self.candidate_list[r].index(city)
      t.append(self.pheromone[r][i] * (self.eta[r][i]**self.beta))

    q = np.random.random(1)[0] # Compute a random q
    # If q is less than q0 then do exploitation otherwise exploration
    if q <= q0:
      arg_max = max(t)
      index = t.index(arg_max)
      s = city_to_visit[index]
    else:
      S = t / sum(t)
      s = np.random.choice(city_to_visit, size = 1, p=S)[0]
    return s

  def length_tour(self, ant):
    length = 0
    tour = self.tour[ant]
    for i in range(0, self.n):
      length += self.dist_mat[tour[i]][tour[i+1]]

    return length

In [ ]:
def acs_for_tsp(acs, init_pos, seeds, q0, method=''):
  epochs = time.time() + 180
  np.random.seed(seeds)

  res = []
  best_tour_len = np.inf
  best_tour = []
  all_tour_length = []
  while time.time() < epochs:

    acs.tour = {i: [] for i in range(acs.m)}
    acs.position = init_pos
    for ant in range(acs.m):
      acs.tour[ant].append(acs.position[ant])

    for city in range(acs.n):
      for ant in range(acs.m):
        from_node = acs.position[ant]
        if city < (acs.n - 1):
          to_node = acs.state_transition_rule(ant, from_node, q0)
        else:
          to_node = acs.tour[ant][0]

        acs.tour[ant].append(to_node)
        acs.position[ant] = to_node
        from_index = acs.candidate_list[to_node].index(from_node)
        to_index = acs.candidate_list[from_node].index(to_node)
        acs.pheromone[from_node][to_index] = acs.local_updating_rule(from_node, to_index)
        acs.pheromone[to_node][from_index] = acs.local_updating_rule(to_node, from_index)

    lt = []

    for ant in range(acs.m):
      tour_length = acs.length_tour(ant)
      lt.append(tour_length)
      all_tour_length.append(tour_length)

    l_gb = min(lt)
    index_l_gb = lt.index(l_gb)

    res.append(l_gb)
    if method == '2opt':
      tour, l_gb = twoOpt_with_cl(acs.tour[index_l_gb], l_gb, acs.dist_mat)
      acs.tour[index_l_gb] = tour.tolist()
    elif method == '2opt_cl':
      new_dict = {}
      for i in acs.candidate_list:
        new_dict[i] = acs.candidate_list[i][:acs.cl]
      tour, l_gb = twoOpt_with_cl(acs.tour[index_l_gb], l_gb, acs.dist_mat, new_dict)
      acs.tour[index_l_gb] = tour.tolist()

    if best_tour_len > l_gb:
      best_tour_len = l_gb
      best_tour = acs.tour[index_l_gb]

    for i in range(0, acs.n-1):
      for j in range(i+1, acs.n):
        acs.global_updating_rule(i, j, best_tour_len, best_tour)

  return res, all_tour_length

In [ ]:
def compute_relative_error(results, ic):
  rel_err = []
  for i in results:
    rel_err.append((i - ic.best_sol)/ic.best_sol)
  return rel_err

def average_run(tour):
  avg_run = []
  s = 0
  for i in range(0, len(tour)):
    if i % ACS.m == 0 and i != 0:
      avg_run.append(s/ACS.m)
      s = 0
    s += tour[i]
  return avg_run

def union_seed(first, second, third):
  res = []
  length = max(len(first), len(second), len(third))
  for i in range(0, length):
    temp = []
    if i < len(first):
      first_seed = first[i]
      temp.append(first_seed)
    if i < len(second):
      second_seed = second[i]
      temp.append(second_seed)
    if i < len(third):
      third_seed = third[i]
      temp.append(third_seed)
    avg = np.mean(temp)
    res.append(avg)
  return res

seeds = [1, 123, 333]

ic_76 = TSP_Instance_Creator("standard", 'eil76.tsp')
ic_130 = TSP_Instance_Creator("standard", 'ch130.tsp')
ic_198 = TSP_Instance_Creator("standard", 'd198.tsp')

np.random.seed(seeds[0])
init_pos = np.random.choice(ic_76.nPoints, size = ACS.m, replace=False)

In [ ]:
acs = ACS(ic)

print('tau0 =', acs.tau0)
for j in acs.candidate_list.keys():
  print()
  print("node          :", j)
  print("candidate list:", acs.candidate_list[j][:3])
  print("eta values    :", acs.eta[j][:3])
  print("pheromone     :", acs.pheromone[j][:3])
  if j>2:
    break


# test twoOpt_with_cl

the implementation of 2opt with the candidate list has worst performances in term of quality but achieves improvements using fewer computation

In [ ]:

ic = TSP_Instance_Creator("standard", 'fl1577.tsp')

initial_sol, initial_len = nn(ic.dist_matrix, starting_node=np.random.choice(ic.nPoints))
acs = ACS(ic)

In [ ]:
def run_acs(seed, q0, method=''):
    np.random.seed(seed)
    init_pos = np.random.choice(ic_76.nPoints, size=ACS.m, replace=False)

    acs = ACS(ic_76)
    res, all_tour = acs_for_tsp(acs, init_pos, seed, q0, method)

    return res, all_tour

def plot_results(re_2opt, re_2opt_cl, avg_run_2opt, avg_run_2opt_cl, label):
    plt.figure(figsize=(25, 5))
    plt.title(f"eil76 - {label}")
    plt.plot(re_2opt, label=f'ACS with 2 Opt, q0 = {label}')
    plt.plot(re_2opt_cl, label=f'ACS with 2 Opt with CL, q0 = {label}')

    plt.axvline(x=re_2opt.index(min(re_2opt)), linestyle=':', label=f'No. of iteration to best tour; q0 = {label}')
    plt.axvline(x=re_2opt_cl.index(min(re_2opt_cl)), linestyle=':', color='r', label=f'No. of iteration to best tour with cl; q0 = {label}')

    plt.legend()
    plt.xlabel('No. of Iterations')
    plt.ylabel('Relative Error')
    plt.show()

    plt.figure(figsize=(25, 5))
    avg_label = ' '.join(label.split('_'))
    plt.title(f"eil76 - Average Cost ({avg_label})")
    plt.plot(avg_run_2opt, label=f'ACS with 2 Opt, q0 = {label}')
    plt.plot(avg_run_2opt_cl, label=f'ACS with 2 Opt with cl, q0 = {label}')

    plt.axhline(y=ic_76.best_sol, color='red', linestyle=':', label='Optimal solution')

    plt.legend()
    plt.xlabel('iterations')
    plt.ylabel('average length of the tour')
    plt.show()

seeds = [seeds[0], seeds[1], seeds[2]]
q0_values = [0.5, 0.98, (1 - 13 / ACS(ic_76).n)]

for seed, q0 in zip(seeds, q0_values):
    res_2opt_76, all_2opt_76 = run_acs(seed, q0, method='2opt')
    res_2opt_cl_76, all_2opt_cl_76 = run_acs(seed, q0, method='2opt_cl')

    re_2opt_76 = compute_relative_error(res_2opt_76, ic_76)
    re_2opt_cl_76 = compute_relative_error(res_2opt_cl_76, ic_76)

    avg_run_2opt_76 = average_run(all_2opt_76)
    avg_run_2opt_cl_76 = average_run(all_2opt_cl_76)

    plot_results(re_2opt_76, re_2opt_cl_76, avg_run_2opt_76, avg_run_2opt_cl_76, str(q0))


In [ ]:
def run_acs(seed, q0, method=''):
    np.random.seed(seed)
    init_pos = np.random.choice(ic_130.nPoints, size=ACS.m, replace=False)

    acs = ACS(ic_130)
    res, all_tour = acs_for_tsp(acs, init_pos, seed, q0, method)

    return res, all_tour

def plot_results(re_2opt, re_2opt_cl, avg_run_2opt, avg_run_2opt_cl, label):
    plt.figure(figsize=(25, 5))
    plt.title(f"ch_130 - {label}")
    plt.plot(re_2opt, label=f'ACS with 2 Opt, q0 = {label}')
    plt.plot(re_2opt_cl, label=f'ACS with 2 Opt with cl, q0 = {label}')

    plt.axvline(x=re_2opt.index(min(re_2opt)), linestyle=':', label=f'No. of iteration to best tour; q0 = {label}')
    plt.axvline(x=re_2opt_cl.index(min(re_2opt_cl)), linestyle=':', color='r', label=f'No. of iteration to best tour with cl; q0 = {label}')

    plt.legend()
    plt.xlabel('No. of Iterations')
    plt.ylabel('Relative Error')
    plt.show()

    plt.figure(figsize=(25, 5))
    avg_label = ' '.join(label.split('_'))
    plt.title(f"ch_130 - Average Cost ({avg_label})")
    plt.plot(avg_run_2opt, label=f'ACS with 2 Opt, q0 = {label}')
    plt.plot(avg_run_2opt_cl, label=f'ACS with 2 Opt with cl, q0 = {label}')

    plt.axhline(y=ic_130.best_sol, color='red', linestyle=':', label='Optimal solution')

    plt.legend()
    plt.xlabel('iterations')
    plt.ylabel('average length of the tour')
    plt.show()

seeds = [seeds[0], seeds[1], seeds[2]]
q0_values = [0.5, 0.98, (1 - 13 / ACS(ic_130).n)]

for seed, q0 in zip(seeds, q0_values):
    res_2opt_130, all_2opt_130 = run_acs(seed, q0, method='2opt')
    res_2opt_cl_130, all_2opt_cl_130 = run_acs(seed, q0, method='2opt_cl')

    re_2opt_130 = compute_relative_error(res_2opt_130, ic_130)
    re_2opt_cl_130 = compute_relative_error(res_2opt_cl_130, ic_130)

    avg_run_2opt_130 = average_run(all_2opt_130)
    avg_run_2opt_cl_130 = average_run(all_2opt_cl_130)

    plot_results(re_2opt_130, re_2opt_cl_130, avg_run_2opt_130, avg_run_2opt_cl_130, str(q0))


In [ ]:
def run_acs(seed, q0, method=''):
    np.random.seed(seed)
    init_pos = np.random.choice(ic_198.nPoints, size=ACS.m, replace=False)

    acs = ACS(ic_198)
    res, all_tour = acs_for_tsp(acs, init_pos, seed, q0, method)

    return res, all_tour

def plot_results(re_2opt, re_2opt_cl, avg_run_2opt, avg_run_2opt_cl, label):
    plt.figure(figsize=(25, 5))
    plt.title(f"d198 - {label}")
    plt.plot(re_2opt, label=f'ACS with 2 Opt, q0 = {label}')
    plt.plot(re_2opt_cl, label=f'ACS with 2 Opt with cl, q0 = {label}')

    plt.axvline(x=re_2opt.index(min(re_2opt)), linestyle=':', label=f'No. of iteration to best tour; q0 = {label}')
    plt.axvline(x=re_2opt_cl.index(min(re_2opt_cl)), linestyle=':', color='r', label=f'No. of iteration to best tour with cl; q0 = {label}')

    plt.legend()
    plt.xlabel('No. of Iterations')
    plt.ylabel('Relative Error')
    plt.show()

    plt.figure(figsize=(25, 5))
    avg_label = ' '.join(label.split('_'))
    plt.title(f"d198 - Average Cost ({avg_label})")
    plt.plot(avg_run_2opt, label=f'ACS with 2 Opt, q0 = {label}')
    plt.plot(avg_run_2opt_cl, label=f'ACS with 2 Opt with cl, q0 = {label}')

    plt.axhline(y=ic_198.best_sol, color='red', linestyle=':', label='Optimal solution')

    plt.legend()
    plt.xlabel('iterations')
    plt.ylabel('average length of the tour')
    plt.show()

seeds = [seeds[0], seeds[1], seeds[2]]
q0_values = [0.5, 0.98, (1 - 13 / ACS(ic_198).n)]

for seed, q0 in zip(seeds, q0_values):
    res_2opt_198, all_2opt_198 = run_acs(seed, q0, method='2opt')
    res_2opt_cl_198, all_2opt_cl_198 = run_acs(seed, q0, method='2opt_cl')

    re_2opt_198 = compute_relative_error(res_2opt_198, ic_198)
    re_2opt_cl_198 = compute_relative_error(res_2opt_cl_198, ic_198)

    avg_run_2opt_198 = average_run(all_2opt_198)
    avg_run_2opt_cl_198 = average_run(all_2opt_cl_198)

    plot_results(re_2opt_198, re_2opt_cl_198, avg_run_2opt_198, avg_run_2opt_cl_198, str(q0))


In [ ]:
def create_acs_dataframe(instances, res_2opt, res_2opt_cl, avg_run_2opt, avg_run_2opt_cl, re_2opt, re_2opt_cl, q0_values):
    rows = []

    for instance in instances:
        for q0 in q0_values:
            rows.append([
                instance.name,
                q0,
                '2opt',
                min(res_2opt),
                np.mean(avg_run_2opt),
                np.std(avg_run_2opt),
            ])
            rows.append([
                instance.name,
                q0,
                '2opt with CL',
                min(res_2opt_cl),
                np.mean(avg_run_2opt_cl),
                np.std(avg_run_2opt_cl),
            ])

    columns = ['Problem', 'q0', 'Variant', 'Best length', 'Average', 'Std.Dev']

    df_acs = pd.DataFrame(rows, columns=columns)
    return df_acs

# Example usage
pd.options.display.float_format = '{:.2f}'.format

instances = [ic_76, ic_130, ic_198]
q0_values = [0.5, 0.98, 1 - 13/76]

df_acs = create_acs_dataframe(instances, res_2opt_76, res_2opt_cl_76, avg_run_2opt_76, avg_run_2opt_cl_76, re_2opt_76, re_2opt_cl_76, q0_values)
df_acs = create_acs_dataframe(instances, res_2opt_130, res_2opt_cl_130, avg_run_2opt_130, avg_run_2opt_cl_130, re_2opt_130, re_2opt_cl_130, q0_values)
df_acs = create_acs_dataframe(instances, res_2opt_198, res_2opt_cl_198, avg_run_2opt_198, avg_run_2opt_cl_198, re_2opt_198, re_2opt_cl_198, q0_values)

pd.set_option('display.expand_frame_repr', False)
print(df_acs)


In [ ]:
start = time()
tour, len_new = twoOpt_with_cl(initial_sol, initial_len, ic.dist_matrix, acs.candidate_list)
print(f' 2opt with candidate: initial len {initial_len}, final len {len_new} \n execution time: {time() - start}')

start = time()
tour, len_new = twoOpt(initial_sol, initial_len, ic.dist_matrix)
print(f' 2opt: initial len {initial_len}, final len {len_new} \n execution time: {time() - start}')

In [ ]:
list_time = []
for _ in range(5):
  initial_sol, initial_len = nn(ic.dist_matrix, starting_node=np.random.choice(ic.nPoints))
  start = time()
  _ = twoOpt_with_cl(initial_sol, initial_len, ic.dist_matrix, acs.candidate_list)
  list_time.append(time()- start)

print(f"mean {np.mean(list_time)},  std {np.std(list_time)}" )

In [ ]:

list_time = []
for _ in range(5):
  initial_sol, initial_len = nn(ic.dist_matrix, starting_node=np.random.choice(ic.nPoints))
  start = time()
  _ = twoOpt(initial_sol, initial_len, ic.dist_matrix)
  list_time.append(time() - start)

print(f"mean {np.mean(list_time)},  std {np.std(list_time)}" )